<div style="border: 1px solid #CCCCFF; border-radius: 1px;">
<div style="margin: 5px">
<b class="alert-heading">Комментарий ревьюера</b>
<p>Вадим, привет!</p>
<p>Меня зовут Алексей Секоцкий. Поздравляю с подготовкой очередного проекта. Предлагаю обращаться друг к другу на «ты» если нет возражений. Ниже стандартный блок с условными обозначениями:</p>
<div class="alert alert-success" role="alert">
всё отлично
</div>
<div class="alert alert-warning" role="alert">
рекомендации на будущее (не требующие доработки проекта)
</div>
<div class="alert alert-danger" role="alert">
критичные моменты требующие внимания (доработки)
</div>
<div class="alert alert-info">
информационные комментарии и итоговый
</div>

<hr>
<p>Пожалуйста, не изменяй и не удаляй мои комментарии – они потребуются для повторной проверки (при необходимости). Задавай вопросы и описывай сделанные изменения, помечая их любым удобным наглядным способом.</p>

# Определение возраста покупателей

Описание проекта:

Сетевой супермаркет «Хлеб-Соль» внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:
Анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
Контролировать добросовестность кассиров при продаже алкоголя.

Необходимо построить модель, которая по фотографии определит приблизительный возраст человека. В  распоряжении набор фотографий людей с указанием возраста.

## Исследовательский анализ данных

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import plotly.express as px
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Загрузим датасет

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345) 

In [ ]:
labels.info()

Датасет содержит 7591 изображение людей с индикатором точного возраста

In [ ]:
labels.head()

In [ ]:
labels.hist(column='real_age',bins=100, range=(0,100));

<div class="alert alert-warning">
<b>Комментарий от ревьюера</b>

У нас возраст от 1 до 100, поэтому и корзин лучше 100 чтобы увидеть особенности распределения и потом в выводе его правильно описать. 50 корзин тоже достаточно, но пики на круглые даты менее заметны.
</div>

<div class="alert alert-block alert-info">
<b>Исправил</b>
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера 2</b>

👍
</div>

Наибольшее количество изображений людей в интервале 18-35 лет

In [ ]:
features, target = next(train_gen_flow)

Взглянем на фотографии людей и их возраст

In [ ]:
fig = plt.figure(figsize=(10,10))
for i in range(10):    
    fig.add_subplot(5, 5, i+1)
    plt.imshow(features[i])
    plt.title(target[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    

<div class="alert alert-warning">
<b>Комментарий от ревьюера</b>

Желательно фото вывести в несколько рядов для компактности и вместо 10 фото лучше 20 чтобы увидеть больше разных вариаций. Также можно добавить вывод возраста над каждым фото для визуальной оценки используя `plt.title()`.
</div>

<div class="alert alert-block alert-info">
<b>Исправил</b>
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера 2</b>

Супер! Учень удобно.
</div>

<div class="alert alert-danger">
<b>Комментарий от ревьюера</b>

По результатам анализа датасета, гистограммы и фото необходимы следующие уточнения:

1. Пояснения по размеру выборки (большая, маленькая, достаточная) и предложения по работе с датасетом такого размера.
2. Выводы по гистограмме. Обратить внимание на выбросы, пики в круглые даты и выдвинуть предположения почему они могли появиться.
3. По фото. Описать особенности увиденные на фото: цветность, качество, размер лиц, угол поворота и т.п. и предложить аугментацию (при необходимости).
<br/>
</div>

<div class="alert alert-block alert-info">
<b>Исправил</b>
</div>

<div class="alert alert-block alert-info">



По результатам анализа датасета были сделаны следующие выводы:
1. Предоставленный датасет можно отнести к средним размерам. В категории лиц 13-25 лет датасет стоит расширить для качественного обучения модели, если одной из наших задач является идентификация лица, которому можно/нельзя продавать алкоголь.
2. В построенной гистограмме есть явные выбросы, например 25, 30, 35, 40, 50, 60, 80 лет. Связаны выбросы, вероятно, с округлениев возраста в результате проводимого опроса. Как правило округление возраста происходит в меньшую сторону.
3. Качество фото в датасете не идеальное. Существуют искажения изображений в виде "растянутости", некоторые фото повернуты на угол до 20 градусов с черными границами, есть черно- белые изображения. 
Как правило, лица занимают большую часть кадра, но в большинстве случаев люди сфотографированы не фронтально, а сбоку. В виду этого стоит ввести horizontal_flip для большего разнообразия изображений и лучшего обучения модели. 
    
    </div>

<div class="alert alert-success">
<b>Комментарий ревьюера</b>

Верно! Пики - это округления при разметке. Аугментация также отличная. Можно еще добавить небольшие вращения и смещения.
</div>

## Обучение модели

```python

import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
 
def load_train(path):
    labels = pd.read_csv(path + 'labels.csv')
    datagen = ImageDataGenerator(validation_split=0.25, rescale=1/255., horizontal_flip=True)
 
    train_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path+'final_files/',
        x_col='file_name',
        y_col='real_age',        
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='training',
        seed=12345)
 
    return train_gen_flow
 
def load_test(path):
    labels = pd.read_csv(path+'labels.csv')
    test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)
 
    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='validation',
        seed=12345)
 
    return test_gen_flow
 
def create_model(input_shape):
    backbone = ResNet50(input_shape=input_shape, weights='imagenet', include_top=False)
    
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(units=1, activation='relu'))
    
    optimizer = Adam(lr=0.0009)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    return(model)
 
def train_model(model, train_data, test_data, batch_size=None, epochs=25, steps_per_epoch=None, validation_steps=None):
    
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)
    
    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
    return model

```

<div class="alert alert-warning">
<b>Комментарий ревьюера</b>

Какая функция активации использовалась? В <a href="https://keras.io/api/layers/core_layers/dense/">документации</a> сказано, что если не задана функция активации, то по умолчанию она линейна. Рекомендую <a href="https://neurohive.io/ru/osnovy-data-science/activation-functions/">статью</a> про функции активации нейросети.
<hr>

Отлично, что используется аугментация horizontal_flip. Другие примеры можно посмотреть в <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator">документации</a> или в <a href="https://towardsdatascience.com/machinex-image-data-augmentation-using-keras-b459ef87cd22">статье</a>.
</div>

<div class="alert alert-block alert-info">
<b>Исправил</b>
</div>

```
Train for 356 steps, validate for 119 steps
Epoch 1/25
2021-09-09 20:57:56.128643: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-09-09 20:58:01.050001: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
356/356 - 125s - loss: 251.6674 - mae: 12.1303 - val_loss: 333.1984 - val_mae: 13.5425
Epoch 2/25
356/356 - 38s - loss: 176.4401 - mae: 10.1939 - val_loss: 289.8830 - val_mae: 12.9012
Epoch 3/25
356/356 - 38s - loss: 149.6896 - mae: 9.3940 - val_loss: 370.4234 - val_mae: 15.1151
Epoch 4/25
356/356 - 39s - loss: 130.3740 - mae: 8.7432 - val_loss: 129.4438 - val_mae: 9.1407
Epoch 5/25
356/356 - 39s - loss: 116.9252 - mae: 8.2403 - val_loss: 138.8261 - val_mae: 8.8689
Epoch 6/25
356/356 - 39s - loss: 101.6428 - mae: 7.6962 - val_loss: 244.3463 - val_mae: 11.4223
Epoch 7/25
356/356 - 38s - loss: 96.0290 - mae: 7.5146 - val_loss: 514.7434 - val_mae: 18.1448
Epoch 8/25
356/356 - 39s - loss: 85.2892 - mae: 7.0706 - val_loss: 132.6884 - val_mae: 8.6782
Epoch 9/25
356/356 - 39s - loss: 75.2064 - mae: 6.6712 - val_loss: 113.0723 - val_mae: 8.1092
Epoch 10/25
356/356 - 39s - loss: 67.0338 - mae: 6.2820 - val_loss: 111.1809 - val_mae: 8.2172
Epoch 11/25
356/356 - 39s - loss: 57.7781 - mae: 5.8838 - val_loss: 116.7400 - val_mae: 8.0286
Epoch 12/25
356/356 - 38s - loss: 54.4810 - mae: 5.6958 - val_loss: 236.6950 - val_mae: 11.7176
Epoch 13/25
356/356 - 39s - loss: 46.0825 - mae: 5.2491 - val_loss: 136.0214 - val_mae: 8.9079
Epoch 14/25
356/356 - 38s - loss: 43.9827 - mae: 5.1589 - val_loss: 114.4340 - val_mae: 8.0582
Epoch 15/25
356/356 - 39s - loss: 37.3860 - mae: 4.7869 - val_loss: 115.8471 - val_mae: 8.0811
Epoch 16/25
356/356 - 39s - loss: 32.5924 - mae: 4.4232 - val_loss: 127.7810 - val_mae: 8.4615
Epoch 17/25
356/356 - 38s - loss: 34.1363 - mae: 4.4992 - val_loss: 107.0655 - val_mae: 7.8909
Epoch 18/25
356/356 - 39s - loss: 31.0435 - mae: 4.3073 - val_loss: 132.2309 - val_mae: 9.1051
Epoch 19/25
356/356 - 39s - loss: 30.5472 - mae: 4.2572 - val_loss: 105.9296 - val_mae: 7.8315
Epoch 20/25
356/356 - 39s - loss: 25.4405 - mae: 3.9081 - val_loss: 102.3604 - val_mae: 7.7243
Epoch 21/25
356/356 - 40s - loss: 23.0493 - mae: 3.7161 - val_loss: 106.1661 - val_mae: 7.6770
Epoch 22/25
356/356 - 39s - loss: 19.9072 - mae: 3.4614 - val_loss: 101.7088 - val_mae: 7.4483
Epoch 23/25
356/356 - 39s - loss: 22.2707 - mae: 3.6413 - val_loss: 152.4810 - val_mae: 9.2975
Epoch 24/25
356/356 - 39s - loss: 23.8701 - mae: 3.7821 - val_loss: 128.9318 - val_mae: 8.4514
Epoch 25/25
356/356 - 39s - loss: 20.9267 - mae: 3.5141 - val_loss: 99.3475 - val_mae: 7.4452
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
119/119 - 10s - loss: 99.3475 - mae: 7.4452
Test MAE: 7.4452
```

<div class="alert alert-warning">
<b>Комментарий ревьюера</b>

Результат для 15 эпох не очень хороший. Возможно это связано с тем, что не задана функции активации. Переучивать модель в тренажере сложно и обычно это делается через техподдержку, поэтому можно оставить текущий вывод.
</div>

<div class="alert alert-block alert-info">

Модель я все-таки переобучил) 

Активацию в модель ввел, переобучил на 25 эпохах. 
Когда активации не было (т.е. линейная) было MAE 7,98 на 15 эпохах, с 'relu' на 25 эпохах стало 7.4452.
    
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера 2</b>

Отлично, что переобучил, как в прямом, так и в переносном смысле :) Обрати внимание на большую разницу между тестовыми и тренировочными значениями метрик. Это говорит о склонности модели к переобучению. Можно это отметить в выводе. В данном проекте оно почти всегда возникает из-за особенностей данных.
</div>

## Анализ обученной модели

В предоставленном датасете содержится 7591 изображение людей с индикатором точного возраста.
Наибольшее количество изображений людей в интервале 18-35 лет.

В обучении была использована предобученная модель ResNet50 с оптимизатором Adam и learning rate 0.0009. По завершении 25 эпох обучения, модель показала MAE 7.44 на тестовой выборке.

Если стоит задача контроля продажи спиртного, то, возможно, датасет нужно расширить фотографиями молодых людей от 13 до 25 лет и обучить модель еще раз для лучшего показателя эффективности модели в данной "группе риска".

<div class="alert alert-info">
<b>Итоговый комментарий ревьюера</b>

Отличный проект и вывод. Остались несколько моментов: вывод по 1 разделу и ответить на вопрос про активацию. Буду ждать новую версию.
<hr>
</div>

<div class="alert alert-info">
<b>Итоговый комментарий ревьюера 2</b>

Поздравляю с успешной подготовкой проекта. Данная тема достаточно обширная для одного спринта. Перед нами стояла задача понять основы для более глубокого погружения. Рекомендую ознакомиться с фреймворком <a href="https://pytorch.org/">PyTorch</a>. Желаю успехов в финальном спринте и профессии!

<hr>
</div>

## Чек-лист

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Исследовательский анализ данных выполнен
- [x]  Результаты исследовательского анализа данных перенесены в финальную тетрадь
- [x]  MAE модели не больше 8
- [x]  Код обучения модели скопирован в финальную тетрадь
- [x]  Результат вывода модели на экран перенесён в финальную тетрадь
- [x]  По итогам обучения модели сделаны выводы